In [1]:
from autogen import AssistantAgent, GroupChatManager, GroupChat
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent, TEXT_FORMATS

from CypherExecutor import CypherCodeExecutor

In [2]:
#TODO: future config_file should be in a json file to not repeat this lines
# This list will 'connect' to the llama3.1 model running on llm container
config_list = [{
        "model": "llama3.1", # The name of the used model (to run the code below it must support the using of tools!)
        "base_url": "http://localhost:11434/v1",
        "api_key": "ollama" # Asked but not used
}]

llm_config = {"config_list": config_list, "temperature": 0.2}

prompt = """You are a data scientist that works with Cypher queries.
All you have to do is translate the given answer as Cypher queries.

You have to respect this rules:
- You must only use the pdf files. 
- You must generate the easiest query possible as markdown block.
- You must instert all the information you have and the page you found them.
- You must be precise. 
- You must re-generate the query if it does not pass the tests.

If the error is not about the query itself you can say 'TERMINATE'.
When you finish your task you can say 'TERMINATE'.
"""

print(f'Accepted formats for "docs_path": \n{TEXT_FORMATS}')

Accepted formats for "docs_path": 
['txt', 'json', 'csv', 'tsv', 'md', 'html', 'htm', 'rtf', 'rst', 'jsonl', 'log', 'xml', 'yaml', 'yml', 'pdf']


In [3]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

doc_retriever = RetrieveUserProxyAgent(
    name="doc_retriever",
    is_termination_msg=termination_msg,
    max_consecutive_auto_reply=3,
    human_input_mode="NEVER",
    system_message="You are an assistant that will pass the pdf manual to the other agents.",
    retrieve_config={
        "task": "code",
        "docs_path": ["https://s3.amazonaws.com/artifacts.opencypher.org/openCypher9.pdf", "https://we-yun.com/doc/neo4j-doc/neo4j-pdf/neo4j-driver-manual-4.4-python.pdf"],
        "chunk_token_size": 1000,
        "model": config_list[0]["model"],
        "get_or_create": True,
    },
    code_execution_config=False
)

coder    = AssistantAgent(
    name="coder",
    is_termination_msg=termination_msg,
    system_message=prompt,
    llm_config=llm_config,
)

feedback = AssistantAgent(
    name="feedback",
    is_termination_msg=termination_msg,
    llm_config=llm_config,
    system_message="""You must review the generated queries based on the test result.""",
)

tester   = AssistantAgent(
    name="tester",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    code_execution_config={"executor": CypherCodeExecutor()}
    )


PROBLEM  = "QUESTION:\nWhat is the smallest path from an actor to another?"

def _reset_agents():
    doc_retriever.reset()
    coder.reset()
    feedback.reset()
    tester.reset()


def rag_chat():
    _reset_agents()
    groupchat = GroupChat(agents=[doc_retriever, coder, tester, feedback], messages=[], 
                          max_round=5, speaker_selection_method="round_robin")
    manager   = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with boss_aid as this is the user proxy agent.
    doc_retriever.initiate_chat(manager,message=PROBLEM,n_results=3)


/home/cate/Scrivania/scuola/uni/traineeship/madt-mlagents/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/cate/Scrivania/scuola/uni/traineeship/madt-mlagents/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
rag_chat()

doc_retriever (to chat_manager):

QUESTION:
What is the smallest path from an actor to another?

--------------------------------------------------------------------------------

Next speaker: coder

[autogen.oai.client: 09-05 16:23:48] {329} WARNING - Model llama3.1 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
coder (to chat_manager):

# Smallest Path Query
## From: [Page 12 of Neo4j Graph Database Documentation](https://neo4j.com/docs/graph-database-manual/current/query-structures/)
### Query:
```cypher
MATCH p=shortestPath((a:Actor)-[*]-(b))
RETURN p;
```
This query uses the `shortestPath` function to find the shortest path between two actors. The `[*]` denotes any type of relationship, and the `p` variable is used to store the result.

Please note that this query assumes that there are relationships between actors in your graph database. If not, you might need to adjust the 

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: Actor)} {position: line: 1, column: 25, offset: 24} for query: 'MATCH p=shortestPath((a:Actor)-[*]-(b))\nRETURN p;'


tester (to chat_manager):

exitcode: 1 (execution failed)
Code output: 

--------------------------------------------------------------------------------

Next speaker: feedback

[autogen.oai.client: 09-05 16:27:17] {329} WARNING - Model llama3.1 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
feedback (to chat_manager):

Based on the provided information, it seems like the query execution failed with exit code 1. To troubleshoot this issue, let's review the generated queries:

1. **Syntax Check**: The Cypher query syntax looks correct. There are no obvious errors in the `MATCH` clause or the `RETURN` statement.
2. **Graph Data**: As mentioned in the note, the query assumes that there are relationships between actors in your graph database. If this is not the case, you might need to add nodes and relationships before running this query.
3. **Shortest Path Function**: The `shortestP